In [ ]:
# default_exp atmosphere

In [ ]:
import logging as log

In [ ]:
import toml
import healpy as hp
import numpy as np
import h5py
from path import Path

In [ ]:
from s4refsimtool.core import get_telescope

In [ ]:
config = toml.load("s4_reference_design.toml")

In [ ]:
base_folder = "/global/cscratch1/sd/keskital/s4sim/reference_tool/out/00000000"

In [ ]:
%ls $base_folder/*atmo*

In [ ]:
log.basicConfig(level=log.INFO)

In [ ]:
# export

def get_telecope_years(config_telescopes, site, channel):
    """Compute the number of telescope/years in the CMB-S4 configuration
    
    config_telescopes : dict
        CMB-S4 telescopes configuration,
        generally config["telescopes"] if loaded from a TOML file
    site : str
        'Pole' or 'Chile', case doesn't matter
    channel : str
        Channel tag, e.g. 'MFHS1'
    """
    telescope_years = 0
    for tube, bands in config_telescopes[get_telescope(channel)].items():
        if bands["site"].lower() == site.lower():
            has_band = bands.get(channel[:-1], 0) > 0
            telescope_years += has_band * bands.get("years", config["experiment"]["total_experiment_length_years"])
    return telescope_years

In [ ]:
s4 = h5py.File("cmbs4_tophat.h5", 'r')

In [ ]:
!tail -30 s4_reference_design.toml

In [ ]:
for site in ["Pole", "Chile"]:
    for channel in s4.keys():
        telescope_years = get_telecope_years(config["telescopes"], site, channel)
        print(site, channel, telescope_years)
        telescope = get_telescope(channel)
        if site == "Chile":
            if telescope == "SAT":
                assert telescope_years == 0, "All SAT at Pole"
            elif channel.startswith("ULFL"):
                assert telescope_years == 0, "No ULFL in Chile"              
            else:
                assert telescope_years == 14, "2 LAT each band"
        if site == "Pole":
            if telescope == "SAT":
                assert telescope_years == 14, "2 SAT telescopes each band"
            else:
                assert telescope_years == 7, "1 LAT with all 4 bands"            

In [ ]:
# export

def load_atmosphere(config, site, channel):
    """Load foreground maps for a channel
    
    Parameters
    ----------
    config : dict
        CMB-S4 configuration,
        generally loaded from a TOML file
    site : str
        'Pole' or 'Chile', case doesn't matter
    channel : str
        Channel tag, e.g. 'MFHS1'
        
    Returns
    -------
    output_map : numpy array
        Output map with all emissions combined, uses nan for missing pixels
    """
    telescope = get_telescope(channel)
    nside = 512 if telescope == "SAT" else 4096
    npix = hp.nside2npix(nside)
    atmosphere_map = hp.read_map(
            Path(base_folder) / f"{site.lower()}_atmosphere_{telescope}_{channel}_filtered_telescope_all_time_all_bmap.fits",
        (0,1,2), dtype=np.float32, verbose=False)
    atmosphere_map[atmosphere_map == hp.UNSEEN] = np.nan
    # input map is 10 days at 100% efficiency
    atmosphere_map *= 10 / (365.25 * config["experiment"]["observing_efficiency"])
    atmosphere_map *= get_telecope_years(config["telescopes"], site, channel)
    return atmosphere_map

In [ ]:
ch = "LFS1"
site = "Pole"

In [ ]:
output_map = load_atmosphere(config, site, channel)

In [ ]:
assert np.nanmax(output_map) < 1e-2 and np.nanmax(output_map) > 1e-3, \
    "Amplitude check failed"

In [ ]:
assert np.nanmin(output_map) > -1e-2 and np.nanmin(output_map) < 1e-3, \
    "Amplitude check failed"

In [ ]:
%matplotlib inline

In [ ]:
hp.mollview(output_map[0], min=-1e-3, max=1e-3, unit="K", title="Atmosphere")

In [ ]:
hp.mollview(output_map[1], min=-1e-7, max=1e-7, unit="K", title="Atmosphere Q")

In [ ]:
hp.mollview(output_map[2], min=-1e-7, max=1e-7, unit="K", title="Atmosphere U")